# 01 - Basic Usage of Qurry

In Qurry, we use the instance of `Qurry` to store Quantum Circuit, experiment, build experiment, and excute them by local simulator or pending to the real quantum machine on `IBM`. In this chapter (or we say this jupyter file), we will introduce how to use Qurry to build a simple quantum circuit and measure their Renyi Entropy by local simulator.


If you are using Colab, you can run the following command in terminal to install Qurry.

```bash
pip install -i https://test.pypi.org/simple/ qurry
```

## 1.1 Setup Environment and Creat experiment executor

In [1]:
from qurry import EntropyMeasure, BackendWrapper
from qiskit import QuantumCircuit

- Before we start, we need to import the package of Qurry and other packages we need. 

- In Qurry, we provider a simple way to import backend called `BackendWrapper`, it will help us to import the backend we need, if your environment includes the GPU acceleration, `qiskit-aer-gpu`, it will be check the availability to access, if does, the GPU backend will also be available.

- The following chapter, we will told you how to use `BackendWrapper` to import the real backend from your IBM account by `IBMProvider` or `AccountProvider`, and the more powerful class `backendManager` to import the real backend more easily.

- In this example, we will use normal `aer_simulator` as our backend


In [2]:
backend = BackendWrapper()
backend('aer')

AerSimulator('aer_simulator_gpu')

In [3]:
print("| Does we have GPU backend to access:", backend.is_aer_gpu)
try:
    # If you have GPU, you can use this backend.
    print(backend('aer_gpu'))
except:
    pass

| Does we have GPU backend to access: True
aer_simulator_gpu


- Now, initialize our `EntropyMeasure`, we have two methods, `randomized` for Randomized Measure as default, and `hadamard` for Hadamard Test, these are our methods for measurement.

- In following tutorial, we will use `randomized` as our measurement method.

In [4]:
experiment_execution_01 = EntropyMeasure(method='hadamard')
print(f'| The first method is which is `Hadamard Test`".')
experiment_execution_02 = EntropyMeasure()
print(f'| The default method is which is `Randomized Measure` we will use.')

| The first method is which is `Hadamard Test`".
| The default method is which is `Randomized Measure` we will use.


---

## 1.2 Load quantum circuit

- We use `TrivialParamagnet` as our target to measure, and add it to our `Qurry` object, and import from `qurry.case` which there are some simple cases.

In [5]:
from qurry.recipe import TrivialParamagnet, GHZ

In [6]:
sample01 = TrivialParamagnet(8)
print("| trivial paramagnet in 8 qubits:")
print(sample01)
sample02 = GHZ(8)
print("| GHZ in 8 qubits:")
print(sample02)
sample03 = QuantumCircuit(8)
sample03.x(range(0,8,2))
print("| Custom circuit:")
print(sample03)

| trivial paramagnet in 8 qubits:
     ┌───┐
q_0: ┤ H ├
     ├───┤
q_1: ┤ H ├
     ├───┤
q_2: ┤ H ├
     ├───┤
q_3: ┤ H ├
     ├───┤
q_4: ┤ H ├
     ├───┤
q_5: ┤ H ├
     ├───┤
q_6: ┤ H ├
     ├───┤
q_7: ┤ H ├
     └───┘
| GHZ in 8 qubits:
     ┌───┐                                   
q_0: ┤ H ├──■────────────────────────────────
     └───┘┌─┴─┐                              
q_1: ─────┤ X ├──■───────────────────────────
          └───┘┌─┴─┐                         
q_2: ──────────┤ X ├──■──────────────────────
               └───┘┌─┴─┐                    
q_3: ───────────────┤ X ├──■─────────────────
                    └───┘┌─┴─┐               
q_4: ────────────────────┤ X ├──■────────────
                         └───┘┌─┴─┐          
q_5: ─────────────────────────┤ X ├──■───────
                              └───┘┌─┴─┐     
q_6: ──────────────────────────────┤ X ├──■──
                                   └───┘┌─┴─┐
q_7: ───────────────────────────────────┤ X ├
                        

- After we prepare three quantum circuits, we can add them to our `Qurry` object.
The attribute `.waves` of `Qurry` object is a customized `dict`, we can check it by print it.

In [7]:
experiment_execution_02.add(
    wave=sample01,              # The circuit object
    key='TrivialParamagnet',    # The name of the circuit
    replace=False               # True for if you want to replace the circuit with the same name.
)                               # Default is False, so if you add the same name when False, 
                                # it will be added with serial number.
experiment_execution_02.add(sample02, 'GHZ')
experiment_execution_02.add(sample03)
print(experiment_execution_02.waves)

<WaveContainer={
    TrivialParamagnet: ...
    GHZ: ...
    2: ...
} with 3 waves load, a customized dictionary>


If you do not give the name of circuit, it will be named by series number.

Now, waves are loading, time to excute.

## 1.3 Excute Circuits

- We can use `.measure()` to excute the specific circuit in `EntropyMeasure` object, and it will return hash id of this experiment and save a `Experiment` object in `EntropyMeasure`.
- For `EntropyMeasure`, it will require the following parameters:
  - wave: the name of circuit in `Qurry.waves` object
  - times: the number of random measurement
  - shots: the number of shots for each measurement. Default is 1024

In [8]:
exp1 = experiment_execution_02.measure(
    wave='TrivialParamagnet',
    times=100,
    shots=1024,
)
exp1 

'4625b1b3-b9d2-4103-922e-adba0d133661'

- And we can check the attribute `.exps` of `EntropyMeasure` object, it will return the hash id of all experiments.

In [9]:
print(experiment_execution_02.exps)

<ExperimentContainer={
    4625b1b3-b9d2-4103-922e-adba0d133661: ...
} with 1 experiments load, a customized dictionary>


- The following is the other informatiom you can access.

In [10]:
experiment_execution_02.exps[exp1]

<qurrentRandomized.Experiment with exp_id=4625b1b3-b9d2-4103-922e-adba0d133661, EntropyRandomizedArguments(exp_name='w=TrivialParamagnet.with100random.qurrent_haar', times=100, measure=(0, 8), unitary_loc=(0, 8), workers_num=16), Commonparams(exp_id='4625b1b3-b9d2-4103-922e-adba0d133661', wave_key='TrivialParamagnet', shots=1024, backend=<AerSimulator('aer_simulator')>, run_args={}, transpile_args={}, tags=(), default_analysis=[], save_location=PosixPath('.'), filename='', files={}, serial=None, summoner_id=None, summoner_name=None, datetimes={'build': '2024-03-18 14:33:17', 'run': '2024-03-18 14:33:17'}), 0 unused arguments, 6 preparing dates, 2 experiment result datasets, and 0 analysis>

In [11]:
print(experiment_execution_02.exps[exp1].statesheet())

 # qurrentRandomized.Experiment with exp_id=4625b1b3-b9d2-4103-922e-adba0d133661
 - arguments
   - exp_name ---------------- w=TrivialParamagnet.with100random.qurrent_haar
   - times ------------------- 100
   - measure ----------------- (0, 8)
   - unitary_loc ------------- (0, 8)
   - workers_num ------------- 16
 - commonparams
   - exp_id ------------------ 4625b1b3-b9d2-4103-922e-adba0d133661   # This is ID is generated by Qurry which is different from 'job_id' for pending.
   - wave_key ---------------- TrivialParamagnet
   - shots ------------------- 1024
   - backend ----------------- aer_simulator
   - run_args ---------------- {}
   - transpile_args ---------- {}
   - tags -------------------- ()
   - default_analysis -------- []
   - save_location ----------- .
   - filename ---------------- 
   - files ------------------- {}
   - serial ------------------ None
   - summoner_id ------------- None
   - summoner_name ----------- None
   - datetimes --------------- {'build': '2

- Then, there is a experiment completed, and we can calculate the Renyi Entropy by `analyze`.

---

## 1.4 Result

- We want to know the Renyi Entropy of half system. Then we can use `.analyze()` to calculate it and it will store a `Analysis` object in `Experiment`.
- The following is the parameter `analyze` of `EntropyMeasure` required.
  - degree: The size or range of subsystem, if it is a number, it will be the size of subsystem, if it is a tuple, it will be the range of subsystem.

In [12]:
exp1_report1 = experiment_execution_02.exps[exp1].analyze(
    degree=(0, 4)
)
exp1_report2 = experiment_execution_02.exps[exp1].analyze(
    degree=4
)

|  - 00:00 < ?

|  - 00:00 < ?

- The `analyze` will return the result of this analysis.

In [13]:
print(exp1_report1)
print(exp1_report1.header)

<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=1.2001620864868163, entropy=-0.26322926048331696, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-03-18 14:33:19', summoner=None, log={})


- It can also be found in the attribute `.reports` of `Experiment` object.

In [14]:
print(experiment_execution_02.exps[exp1].reports)
print(experiment_execution_02.exps[exp1].reports[0])
print(experiment_execution_02.exps[exp1].reports[0].header)

<AnalysisContainer={
    0: ...
    1: ...
} with 2 analysis load, a customized dictionary>
<qurrentRandomized.Analysis with serial=0, AnalysisInput(degree=(0, 4), shots=1024, unitary_loc=(0, 8)), AnalysisContent(purity=1.2001620864868163, entropy=-0.26322926048331696, and others), 0 unused arguments>
AnalysisHeader(serial=0, datetime='2024-03-18 14:33:19', summoner=None, log={})


- We can use `.export()` to get the result of this analysis.

In [15]:
main, side_prodct = experiment_execution_02.exps[exp1].reports[0].export()

- And there is it

In [16]:
main

{'purity': 1.2001620864868163,
 'entropy': -0.26322926048331696,
 'puritySD': 1.1239147826945266,
 'entropySD': 1.3510394151190361,
 'bitStringRange': [0, 4],
 'allSystemSource': 'independent',
 'purityAllSys': 1.350189094543457,
 'entropyAllSys': -0.4331614714656859,
 'puritySDAllSys': 1.4199841410257523,
 'entropySDAllSys': 1.5172719781828246,
 'bitsStringRangeAllSys': [0, 8],
 'errorRate': -0.16256715294657023,
 'mitigatedPurity': 0.9042384939238135,
 'mitigatedEntropy': 0.14522475961169473,
 'num_qubits': 8,
 'measure': ['measure range:', [0, 8]],
 'measureActually': [0, 8],
 'measureActuallyAllSys': [0, 8],
 'countsNum': 100,
 'takingTime': 0.000836056,
 'takingTimeAllSys': 0.00334642,
 'input': {'degree': [0, 4], 'shots': 1024, 'unitary_loc': [0, 8]},
 'header': {'serial': 0,
  'datetime': '2024-03-18 14:33:19',
  'summoner': None,
  'log': {}}}

- Also, `sideProdct` will record the data of each circuit, and you can use it to do more analysis.

In [17]:
side_prodct

{'purityCells': {87: 0.41632843017578125,
  35: 1.540395736694336,
  99: 0.24810028076171875,
  81: 2.309049606323242,
  15: 1.250265121459961,
  73: 0.21718406677246094,
  10: 0.5714302062988281,
  8: 0.2646770477294922,
  33: 1.3607463836669922,
  12: 0.6284217834472656,
  29: 0.4827556610107422,
  80: 1.7007732391357422,
  30: 1.6592826843261719,
  62: 0.5863933563232422,
  77: 0.24023818969726562,
  3: 0.7625408172607422,
  32: 0.31729698181152344,
  98: 3.317646026611328,
  64: 1.2047576904296875,
  70: 4.04511833190918,
  17: 4.54576301574707,
  67: 0.7181148529052734,
  57: 2.2141780853271484,
  75: 0.2463665008544922,
  54: 1.9171924591064453,
  82: 1.7292346954345703,
  96: 0.5467281341552734,
  91: 0.9411029815673828,
  14: 0.15072250366210938,
  65: 4.197038650512695,
  51: 2.9863052368164062,
  53: 1.4476757049560547,
  68: 0.6845722198486328,
  28: 0.5784397125244141,
  94: 0.3826484680175781,
  69: 0.620330810546875,
  44: 0.9136371612548828,
  86: 1.8575916290283203,
  1

In [18]:
experiment_execution_02.exps[exp1].reports[0].statesheet()

 # qurrentRandomized.Analysis with serial=0
 - header
   - serial ----------------------- 0
   - datetime --------------------- 2024-03-18 14:33:19
   - summoner --------------------- None
   - log -------------------------- {}
 - input
   - degree ----------------------- (0, 4)
   - shots ------------------------ 1024
   - unitary_loc ------------------ (0, 8)
 - outfields -------------------- 0 ........  # Number of unused arguments.
 - content
   - purity ----------------------- 1.2001620864868163
   - entropy ---------------------- -0.26322926048331696
   - puritySD --------------------- 1.1239147826945266
   - entropySD -------------------- 1.3510394151190361
   - purityCells ------------------ {87: 0.41632843017578125, 35: 1.540395736694336, 99: 0.24810028076171875, 81: 2.309049606323242, 15: 1.250265121459961, 73: 0.21718406677246094, 10: 0.5714302062988281, 8: 0.2646770477294922, 33: 1.3607463836669922, 12: 0.6284217834472656, 29: 0.4827556610107422, 80: 1.7007732391357422, 30:

---

## Post-Process Availablities and Version Info

We currently do not support Qsikit 1.0 for we are working on it, and we will support it in the future.

In [19]:
from qurry import __version__
from qurry.process import AVAIBILITY_STATESHEET
print("| Qurry version:", __version__)
print(AVAIBILITY_STATESHEET)

| Qurry version: 0.7.1.dev3
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability           Python Cython Rust  
 - randomized_measure
   - entangled_core ............... True   True   True  
   - purity_cell .................. True   True   True  
   - wavefunction_overlap ......... True   True   None  
   - echo_cell .................... True   True   None  
 - utils
   - construct .................... True   True   True  
   - construct .................... True   None   True  
--------------------------------------------------------
   + True ..... Working normally.
   + False .... Exception occurred.
   + None ..... Not supported yet.
--------------------------------------------------------

